# Video Sampling Example

This notebook demonstrates how to use the sampling functions to generate videos from text prompts.

In [1]:
import sys
from sample import sample_videos, sample_single_video, prompts
from release_server import load_merge_config, load_all

sageattention package is not installed
flash attn 2 available True
flash attn 3 available False
sage attn available False
SAGEATTN_AVAILABLE: False
DO_COMPILE False


In [2]:
import torch
import gc
config = load_merge_config("/workspace/release-sf/configs/self_forcing_server_14b.yaml")
models = load_all(config)
print("✅ Models loaded and ready to use!")
gc.collect()
torch.cuda.empty_cache()


2025-10-09 23:57:50.136 - INFO - Starting model loading...
2025-10-09 23:57:50.137 - DEBUG - Using checkpoint: checkpoints/14b_200.safetensors
Loading transformer:   0%|          | 0/4 [00:00<?, ?it/s]2025-10-09 23:57:59.366 - DEBUG - Transformer import took: 9.23s
2025-10-09 23:58:02.328 - DEBUG - Loading transformer state dict from checkpoints/14b_200.safetensors


Loading WAN model, with name Wan2.1-T2V-14B
timestep shift,  5.0


2025-10-09 23:58:43.042 - DEBUG - Transformer load completed in: 43.68s, total: 52.90s
2025-10-09 23:58:43.044 - DEBUG - Loading transformer took: 52.90s
Loading text encoder:  25%|██▌       | 1/4 [00:52<02:38, 52.91s/it]2025-10-09 23:58:43.045 - DEBUG - Text encoder import took: 0.00s
2025-10-09 23:58:44.910 - DEBUG - Text encoder load completed in: 1.86s, total: 1.86s
2025-10-09 23:58:44.911 - DEBUG - Loading text encoder took: 1.87s
Loading VAE:  50%|█████     | 2/4 [00:54<00:45, 22.88s/it]         2025-10-09 23:58:44.912 - DEBUG - Using demo_utils.vae_block3.VAEEncoderWrapper
2025-10-09 23:58:44.912 - DEBUG - Using demo_utils.vae_block3.VAEDecoderWrapper
2025-10-09 23:58:44.937 - INFO - loading Wan-2.1/Wan2.1-T2V-1.3B/Wan2.1_VAE.pth
2025-10-09 23:58:45.539 - DEBUG - VAE load completed in: 0.63s
2025-10-09 23:58:45.556 - DEBUG - Loading VAE took: 0.64s
Initializing pipeline:  75%|███████▌  | 3/4 [00:55<00:12, 12.73s/it]2025-10-09 23:58:45.587 - DEBUG - Pipeline import took: 0.03s
20

Incompatible _IncompatibleKeys(missing_keys=['mean', 'std'], unexpected_keys=[]) while loading vae decoder
KV inference with 3 frames per block
✅ Models loaded and ready to use!


In [ ]:
import matplotlib.pyplot as plt
from release_server import get_denoising_schedule
from pprint import pprint


models.pipeline.scheduler.set_timesteps(1000, training=True)
st = models.pipeline.scheduler.timesteps

# print(st)
st = st.clone()
zero_padded_timesteps = torch.cat((st.clone().cpu(), torch.tensor([0], dtype=torch.float32, device="cpu"))).to("cuda")
# pprint(zero_padded_timesteps.tolist())
t = get_denoising_schedule(zero_padded_timesteps.cpu(), 1, steps=5)
# print(t)
# print(zero_padded_timesteps[-4:])
# plt.plot(zero_padded_timesteps.tolist())
# # print(type(t))
# t = torch.nn.functional.interpolate(t, size=len(zero_padded_timesteps)).squeeze(0)
# print(t.shape)
print(t)

plt.plot(t.tolist())
plt.show()


In [3]:

test_prompts = [
    "A sorcerer stands with one hand outstretched, holding a roiling flame that coils and twists restlessly around his palm and fingers, sparks shooting off in unpredictable arcs. The fire spirals and lashes outward, wrapping around his arm like a living serpent before snapping back toward his hand in a continuous, fluid motion. His cloak ripples in the heat’s updraft as the flame flares and contracts, creating bursts of glowing embers that rise and scatter through the air",
    "Adrenaline-pumped, wide-eyed ginger kitten in vintage aviator goggles blasts down a narrow cobblestone street on a bright-yellow mini-bicycle. Camera whip-pans in from behind, then snap-zooms past the spinning front wheel into an ultra-close-up of the cat’s determined face, fur rippling in the wind. Hard sunlight streaks across the scene, creating dynamic highlights and streaking motion-blur on the spokes.",
    "A stylish woman walks down a Tokyo street filled with warm glowing neon and animated city signage. She wears a black leather jacket, a long red dress, and black boots, and carries a black purse. She wears sunglasses and red lipstick. She walks confidently and casually. The street is damp and reflective, creating a mirror effect of the colorful lights. Many pedestrians walk about.",
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The static camera holds the scene firmly in place, emphasizing the intensity and grace of his movements against the stillness of the landscape.",
    "Tracking shot, cinematic style, of a surreal alien bird with iridescent feathers and bio-luminescent wings, gracefully flying through a dense alien forest — the camera follows the bird in one continuous dynamic shot, weaving through twisted glowing trees, over bioluminescent mushrooms, and past floating pollen-like orbs. The bird performs elegant aerial maneuvers — barrel rolls, sharp turns, dives — as it dodges branches and interacts with the strange atmosphere. The forest pulses with ambient light and motion-reactive flora.",
    "A single crystalline drop of water hovers in midair against a soft gradient background, shimmering with refracted light. The drop quivers, rippling with internal motion, before stretching outward as if blooming from within. Its surface tension warps into delicate translucent petals that unfurl in one continuous motion. The petals radiate with glistening reflections, the drop gracefully becoming a luminous flower suspended in space.",
    "Two scarred alley cats, one orange tabby with torn ears and one lean gray bruiser, stand upright in a flickering streetlight. They lash out with vicious swipes, their miniature boxing gloves cracked and worn, landing heavy blows that make fur and sweat fly. The gray cat snarls as it takes a hook to the jaw, stumbling back before lunging forward with a brutal counter. The sound of claws scraping the pavement mixes with the thud of their punches, the fight relentless and raw.",
    "She returns in a radiant gown made entirely of flowing gold silk, draped in voluminous layers that trail behind her in shimmering waves. The fabric ripples dramatically with each step, catching and scattering light across the runway. The dress moves like liquid metal, transforming her walk into a continuous display of luxury and opulence, radically different from her previous sculptural look.",
    "Under glowing neon lights in a surreal diner, a polar bear sits awkwardly in a red leather booth, gripping a massive BLT with mayonnaise dripping from its paws. It devours the sandwich with primal hunger, grease smearing across its white fur as the jukebox hums in the background. Fries scatter across the floor as the bear’s huge body shifts against the tiny furniture, the booth creaking under its weight. The continuous shot blends humor, absurdity, and menace in equal measure.",
    "Inside a steaming porcelain coffee cup, two tiny wooden galleons clash on dark, swirling waves of coffee. Cannons fire with sharp flashes, sending miniature plumes of smoke curling into the air, while splinters of wood spray from direct hits. The surface of the coffee sloshes violently with every broadside, waves lapping against the cup’s rim. Steam drifts upward, blending with the smoke, as the ships circle each other in a chaotic, continuous duel.",
    "In a brightly lit gymnasium, a young gymnast in a red leotard swings powerfully on the uneven bar, launching into a perfect backflip. Her body tucks tightly mid-air, spinning in one continuous motion before her hands snap back onto the bar with precision. Chalk dust bursts into the air with the impact, drifting like smoke around her. The crowd gasps faintly in the background, the scene locked in graceful, dynamic motion.",
    "Under the glare of bright stadium lights, the golden retriever launches high off the springboard, twisting into a daring double flip. Its ears flap wildly mid-air, and the shimmering pool reflects its spinning form. The audience erupts in cheers as the dog’s paws extend gracefully in the final moment before the dive. A splash explodes upward as it enters the water, sending ripples across the Olympic rings painted on the pool floor.", 
]

In [ ]:
import sample
from sample import GenerateParams
params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=8,
    seed=1234,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=5,
)
prompts = [
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The static camera holds the scene firmly in place, emphasizing the intensity and grace of his movements against the stillness of the landscape.",
]
results = sample.sample_videos(
    prompts_list=test_prompts,
    models=models,  # Reuse loaded models
    params=params,
    output_dir="current_release_4step",
    save_videos=True,
    fps=16
)

Sampling videos:   0%|          | 0/12 [00:00<?, ?it/s]


📝 Prompt 1/12: A sorcerer stands with one hand outstretched, holding a roiling flame that coils and twists restlessly around his palm and fingers, sparks shooting off in unpredictable arcs. The fire spirals and lashes outward, wrapping around his arm like a living serpent before snapping back toward his hand in a continuous, fluid motion. His cloak ripples in the heat’s updraft as the flame flares and contracts, creating bursts of glowing embers that rise and scatter through the air
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization o

Sampling videos:   8%|▊         | 1/12 [00:15<02:48, 15.36s/it]

✅ Video saved to current_release_4step/prompt_000.mp4

📝 Prompt 2/12: Adrenaline-pumped, wide-eyed ginger kitten in vintage aviator goggles blasts down a narrow cobblestone street on a bright-yellow mini-bicycle. Camera whip-pans in from behind, then snap-zooms past the spinning front wheel into an ultra-close-up of the cat’s determined face, fur rippling in the wind. Hard sunlight streaks across the scene, creating dynamic highlights and streaking motion-blur on the spokes.
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cach

Sampling videos:  17%|█▋        | 2/12 [00:30<02:33, 15.33s/it]

✅ Video saved to current_release_4step/prompt_001.mp4

📝 Prompt 3/12: A stylish woman walks down a Tokyo street filled with warm glowing neon and animated city signage. She wears a black leather jacket, a long red dress, and black boots, and carries a black purse. She wears sunglasses and red lipstick. She walks confidently and casually. The street is damp and reflective, creating a mirror effect of the colorful lights. Many pedestrians walk about.
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent f

## Video to Video

In [ ]:

params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=9,
    seed=1234,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=8,

    input_vid = "......" # ADD this
)
prompts = [
    "A massive wooden ship sails through a dark, stormy sea, its masts swaying violently as enormous waves crash against the hull. The ocean roars with relentless power, sending bursts of white spray over the deck as the crew struggles to keep control. Rain pours down in thick sheets, blurring the horizon into a gray haze. The camera moves with the ship’s motion, capturing the rhythm of the storm and the sheer force of the sea.",
]
results = sample.sample_videos(
    prompts_list=[p],
    models=models,  # Reuse loaded models
    params=params,
    output_dir="out_release",
    save_videos=True,
    fps=16
)